# Asset Class Trend Following 策略回測 (2025-1)

## 全域參數設定 (可在此手動調整策略參數)

In [ ]:
# === 全域參數設定區 ===
SMA_PERIOD = 64        # SMA 週期 (上限 80)
ROC_PERIOD = 23        # ROC 週期 (上限 80)
STOP_LOSS_PCT = 0.09   # 停損比例 (例如 0.09 代表 9.0%)
INITIAL_CAPITAL = 30000000 # 初始資金
DATA_FILE = '個股2.xlsx'    # 資料檔案名稱
# ====================

## 策略邏輯實現

In [ ]:
import pandas as pd
import numpy as np
import xlsxwriter

def clean_data(filepath):
    df_raw = pd.read_excel(filepath, header=None)
    stock_codes = df_raw.iloc[0, 2:].astype(str).values
    stock_names = df_raw.iloc[1, 2:].values
    dates = pd.to_datetime(df_raw.iloc[2:, 1])
    prices = df_raw.iloc[2:, 2:].astype(float)
    prices.index = dates
    prices.columns = stock_codes
    prices = prices.bfill().ffill()
    prices = prices.dropna(axis=1, how='all')
    return prices, dict(zip(stock_codes, stock_names))

class Backtester:
    def __init__(self, prices, initial_capital=30000000):
        self.prices = prices.values
        self.dates = prices.index
        self.assets = prices.columns
        self.initial_capital = initial_capital

    def run(self, sma_period, roc_period, stop_loss_pct):
        prices_df = pd.DataFrame(self.prices, index=self.dates, columns=self.assets)
        sma = prices_df.rolling(window=sma_period).mean().values
        roc = prices_df.pct_change(periods=roc_period).values
        
        capital = self.initial_capital
        portfolio = {} # asset_idx -> info
        equity_curve = np.full(len(self.dates), self.initial_capital, dtype=float)
        rebalance_log = []
        holdings_history = []
        
        pending_sl = set()
        pending_rebalance = None 
        
        start_idx = 68 # Jan 10 (Fri)
        
        for i in range(len(self.dates)):
            date = self.dates[i]
            
            # --- 1. EXECUTION ---
            if i > 0:
                for idx in list(pending_sl):
                    if idx in portfolio:
                        info = portfolio.pop(idx)
                        sell_p = self.prices[i][idx]
                        capital += info['shares'] * sell_p
                        rebalance_log.append({
                            '日期': date, '股票代號': self.assets[idx],
                            '狀態': '賣出', '價格': sell_p, '原因': '停損',
                            '動能值': roc[i-1][idx] if i > 0 else 0
                        })
                pending_sl = set()
                
            if pending_rebalance:
                to_sell, to_keep, to_buy = pending_rebalance
                for idx in to_sell:
                    if idx in portfolio:
                        info = portfolio.pop(idx)
                        sell_p = self.prices[i][idx]
                        capital += info['shares'] * sell_p
                        rebalance_log.append({
                            '日期': date, '股票代號': self.assets[idx],
                            '狀態': '賣出', '價格': sell_p, '原因': '再平衡',
                            '動能值': roc[i-1][idx]
                        })
                for idx in to_keep:
                    if idx in portfolio:
                        rebalance_log.append({
                            '日期': date, '股票代號': self.assets[idx],
                            '狀態': '保持', '價格': self.prices[i][idx], '原因': '趨勢持續',
                            '動能值': roc[i-1][idx]
                        })
                slot_cap = self.initial_capital / 3
                for idx in to_buy:
                    buy_p = self.prices[i][idx]
                    shares = slot_cap // buy_p
                    if shares > 0 and capital >= shares * buy_p:
                        capital -= shares * buy_p
                        portfolio[idx] = {'shares': shares, 'max_price': buy_p, 'buy_price': buy_p, 'buy_date': date}
                        rebalance_log.append({
                            '日期': date, '股票代號': self.assets[idx],
                            '狀態': '買進', '價格': buy_p, '原因': '符合趨勢',
                            '動能值': roc[i-1][idx]
                        })
                pending_rebalance = None

            # --- 2. VALUATION ---
            curr_val = capital
            for idx, info in portfolio.items():
                p = self.prices[i][idx]
                if p > info['max_price']: info['max_price'] = p
                curr_val += info['shares'] * p
            equity_curve[i] = curr_val
            
            # Record daily holdings
            for idx, info in portfolio.items():
                holdings_history.append({'Date': date, 'Asset': self.assets[idx], 'Shares': info['shares'], 'Value': info['shares'] * self.prices[i][idx]})

            # --- 3. SIGNAL GENERATION ---
            for idx, info in portfolio.items():
                if self.prices[i][idx] < info['max_price'] * (1 - stop_loss_pct):
                    pending_sl.add(idx)
            
            if i >= start_idx and (i - start_idx) % 5 == 0:
                elig = (self.prices[i] > sma[i]) & (roc[i] > 0)
                elig_idxs = np.where(elig)[0]
                elig_rocs = roc[i][elig_idxs]
                top_k = min(3, len(elig_idxs))
                target_idxs = elig_idxs[np.argsort(elig_rocs)[-top_k:][::-1]]
                target_set = set(target_idxs)
                curr_set = set(portfolio.keys())
                pending_rebalance = (curr_set - target_set, curr_set & target_set, target_set - curr_set)

        return pd.Series(equity_curve, index=self.dates), pd.DataFrame(rebalance_log), pd.DataFrame(holdings_history)

def calculate_metrics(equity_curve, rebalance_log, start_date='2025-01-01', end_date='2025-12-31'):
    eq = equity_curve[start_date:end_date]
    if eq.empty: return {}
    base_val = 30000000.0
    total_ret = (eq.iloc[-1] / base_val) - 1
    days = (eq.index[-1] - eq.index[0]).days
    cagr = (1 + total_ret) ** (365.25 / days) - 1 if days > 0 else 0
    rolling_max = eq.cummax()
    max_dd = ((eq - rolling_max) / rolling_max).min()
    calmar = cagr / abs(max_dd) if max_dd != 0 else 0
    
    positions = {}
    wins = []
    for _, row in rebalance_log.iterrows():
        ticker = row['股票代號']
        if row['日期'] < pd.to_datetime(start_date): continue
        if row['狀態'] == '買進': positions[ticker] = row['價格']
        elif row['狀態'] == '賣出' and ticker in positions:
            wins.append(row['價格'] > positions[ticker])
            del positions[ticker]
    win_rate = np.mean(wins) if wins else 0
    return {'CAGR': cagr, 'MaxDD': max_dd, 'Calmar': calmar, 'WinRate': win_rate}


## 執行回測與績效分析 (2025)

In [ ]:
prices, code_to_name = clean_data(DATA_FILE)
bt = Backtester(prices, INITIAL_CAPITAL)
eq, log, holdings = bt.run(SMA_PERIOD, ROC_PERIOD, STOP_LOSS_PCT)

def calculate_metrics_2025(equity_curve, rebalance_log):
    start_date, end_date = '2025-01-01', '2025-12-31'
    eq = equity_curve[start_date:end_date]
    base_val = 30000000.0
    total_ret = (eq.iloc[-1] / base_val) - 1
    days = (eq.index[-1] - eq.index[0]).days
    cagr = (1 + total_ret) ** (365.25 / days) - 1
    rolling_max = eq.cummax()
    max_dd = ((eq - rolling_max) / rolling_max).min()
    calmar = cagr / abs(max_dd) if max_dd != 0 else 0
    return {'CAGR': cagr, 'MaxDD': max_dd, 'Calmar': calmar}

metrics = calculate_metrics_2025(eq, log)
print('2025 績效指標：')
for k, v in metrics.items():
    print(f"{k}: {v:.2%}" if k != 'Calmar' else f"{k}: {v:.2f}")

## 參數高原分析 (SMA)

In [ ]:
plateau = []
for sma in [60, 62, 64, 66, 68]:
    eq_p, log_p, _ = bt.run(sma, ROC_PERIOD, STOP_LOSS_PCT)
    m = calculate_metrics_2025(eq_p, log_p)
    plateau.append({'SMA': sma, 'CAGR': f"{m['CAGR']:.2%}", 'MaxDD': f"{m['MaxDD']:.2%}", 'Calmar': f"{m['Calmar']:.2f}"})
import pandas as pd
pd.DataFrame(plateau)